In [ ]:
#import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import numpy as np

In [ ]:
#set path
PATH = "/Users/Immy/Desktop/chromedriver"

In [ ]:
#lists for results:
prices= []
adnames = []
ad_details = []

#text of subcategories
category = []
#numeric placeholders for subcategories
subcat = []

#set url to horsequest advanced search page
driver = webdriver.Chrome(PATH)
url = 'https://www.horsequest.co.uk/search'
driver.get(url)
time.sleep(1)

#define functions

#determine number of pages for the search results, save as an int
def page_counter(x):
    if x == 1:
        return 1
    else:
        lastpagetxt = x.text
        lastpage= int(lastpagetxt.split()[-1])
        return lastpage

#retrieve data on price, title, and details on a single page
def get_data():
    cards = driver.find_elements_by_class_name('card-main')
    
    for card in cards:
        #retrieve advert title
        titles = card.find_elements_by_class_name('card-title')
        for i in titles:
            titletext = i.text
            adnames.append(titletext)
    
    for card in cards:
        #retrieve advert price
        pricelab = card.find_elements_by_class_name('card-price')
        for i in pricelab:
            result = i.text
            prices.append(result)

    #retrieve advert details *if present*
    for card in cards:
        details=card.find_elements_by_class_name("card-breedInfo")
        if len(details) == 1:
            for i in details:
                detailtext = i.text
                ad_details.append(detailtext)
        #if details are absent, append "blank"
        else:
            ad_details.append("BLANK")

    #add a numeric placeholder for the subcategory being scraped 
    for card in cards:
        subcat.append(subcatno)

#go to each page, and then extract data          
def main():
    #select category 'horses for sale'
    dd1 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[(@id = 'ContentPanel_ddlCategory')]/option[text()='Horses For Sale']")
        ))
    dd1.click()

    time.sleep(1)

    #identify the subcategories in the second dropdown, then save them to "options"
    dd2 = Select(driver.find_element_by_xpath("//*[(@id = 'ContentPanel_ddlSubCategory')]"))
    options = dd2.options
    #save the text names of horse types to a list for later use
    for option in options:
        category.append(option.text)

    #iterate through subcats in this dropdown- click each subcat, then click search:
    for item in range(1,len(options)):

        #pick a subcat
        dd2 = Select(driver.find_element_by_xpath("//*[(@id = 'ContentPanel_ddlSubCategory')]"))
        time.sleep(1)
        dd2.select_by_index(item)
        time.sleep(1)

        #scroll to search button
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        #click go
        element = driver.find_element_by_name('ctl00$ContentPanel$btnsubmit')
        actions = ActionChains(driver)
        actions.move_to_element(element).click().perform()

        time.sleep(2)

        #look if there are multiple pages to scrape. if so, calc num of pages. 
        try:
            pagerange= driver.find_element_by_css_selector(".pagination-pagenumber")
            allpages= page_counter(pagerange)
        except NoSuchElementException:
            pagerange = 1   
        subcatno = item
        #extract data
        for j in range(allpages):
            try:
            #if a next page button is present: get data from this page, then click next page 
                nextpage = driver.find_element_by_xpath("//*[(@id = 'ContentPanel_ucTopPagination_hlPageNext')]")
                get_data()
                nextpage.click()
                time.sleep(1)
            #otherwise, get data from this page
            except NoSuchElementException:
                get_data()

                #go back to starter page
                driver.get(url)
                #click first dropdown and re-input horses for sale- this prevents dropdown timeout
                dd1 = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[(@id = 'ContentPanel_ddlCategory')]/option[text()='Horses For Sale']")
                ))

                dd1.click()

                time.sleep(1)
                #open second drop down, save its categories to options- to prevent dropdown timeout
                dd2 = Select(driver.find_element_by_xpath("//*[(@id = 'ContentPanel_ddlSubCategory')]"))
                options = dd2.options
                
main()

driver.quit()

In [ ]:
#to interrupt a scrape midway through
driver.quit()

*** CREATE A DATAFRAME FOR THE DATA***


note- if scrape is interrupted, lists will likely be different lengths and if so, df will not be possible to make

In [ ]:
#make empty df
df= pd.DataFrame()

In [ ]:
#advert title column
df["Name"] = adnames

In [ ]:
#details column
df["Details"]= ad_details

In [ ]:
#price column
df["Price"]= prices

In [ ]:
#numeric subcategory column
df["Category"] = subcat

In [ ]:
#function to return a text equivalent of the numeric subcat
def gettitle(x):
    return category[x - 1] 

In [ ]:
#make text subcategory column eg allrounders
df['Type'] = list(map(gettitle,df["Category"]))

In [ ]:
#head of df
df.head()